# Combining all together

Now we have all the pieces necessary to make implement the interactive optimization process using clustering as surrogates and different scalarization functions. Yeah.

In [1]:
 %matplotlib inline
import seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from ASF import ASF
from gradutil import *
from pyomo.opt import SolverFactory
seedn = 1

First lets take all the data in

In [2]:
%%time
revenue, carbon, deadwood, ha = init_boreal()
n_revenue = nan_to_bau(revenue)
n_carbon= nan_to_bau(carbon)
n_deadwood = nan_to_bau(deadwood)
n_ha = nan_to_bau(ha)
ide = ideal(False)
nad = nadir(False)
opt = SolverFactory('glpk')

In [3]:
x = pd.concat((n_revenue, n_carbon, n_deadwood, n_ha), axis=1)
x_stack = np.dstack((n_revenue, n_carbon, n_deadwood, n_ha))

Normalize all the columns in 0-1 scale

In [4]:
%%time
x_norm = normalize(x.values)
x_norm_stack = normalize(x_stack)

Cluster the data to some clusters and calculate correponding weights

In [5]:
%%time 
nclust1 = 200
c, xtoc, dist = cluster(x_norm, nclust1, seedn, verbose=1)
w = np.array([sum(xtoc == i) for i in range(nclust1)])

Calculate new cluster centers using average from normalized data

In [6]:
c_mean = np.array([x_norm_stack[xtoc == i].mean(axis=0) for i in range(nclust1)])

Calculate new cluster centers using stand closest to the center of the cluster. This option could be more justified in the practical level, because we don't know so much about the forestry and features of forest stands.

In [7]:
c_close = np.array([x_norm_stack[np.argmin(dist[xtoc == i])] for i in range(nclust1)])

Calculate solution for some reference using the cluster centers

In [10]:
ref = np.array((ide[0], 0, 0, 0))
mean_asf = ASF(ide, nad, ref, c_mean, weights=w)
res_asf_mean = opt.solve(mean_asf.model)
mean_stom = ASF(ide, nad, ref, c_mean, weights=w, scalarization='stom')
res_stom_mean = opt.solve(mean_stom.model)
mean_guess = ASF(ide, nad, ref, c_mean, weights=w, scalarization='guess')
res_stom_mean = opt.solve(mean_guess.model)

In [11]:
model_to_real_values(x_stack, xtoc, mean_asf.model)

In [12]:
model_to_real_values(x_stack, xtoc, mean_stom.model)

In [13]:
model_to_real_values(x_stack, xtoc, mean_guess.model)

Calculate solution for some reference using the stands closest to the cluster center

In [14]:
ref = np.array((ide[0], 0, 0, 0))
close_asf = ASF(ide, nad, ref, c_close, weights=w)
res_close = opt.solve(close_asf.model)
close_stom = ASF(ide, nad, ref, c_close, weights=w, scalarization='stom')
res_stom_close = opt.solve(close_stom.model)
close_guess = ASF(ide, nad, ref, c_close, weights=w, scalarization='guess')
res_stom_close = opt.solve(close_guess.model)

In [15]:
model_to_real_values(x_stack, xtoc, close_asf.model)

In [16]:
model_to_real_values(x_stack, xtoc, close_stom.model)

In [17]:
model_to_real_values(x_stack, xtoc, close_guess.model)

It looks like that when using real stands closest to center the solutions are more averaged than when using averages of the clusters. That is quite interesting, because the centroids of the clusters in the k-means method are all real individuals. 